In [9]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras import regularizers
import tensorflow as tf
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


In [6]:
train_path = "dataset_full_en/train"
test_path = "dataset_full_en/test"
validation_path = "dataset_full_en/validation"

In [10]:
num_classes = len(os.listdir(train_path))
num_classes

60

In [11]:
label_list = os.listdir(train_path)
label_list

['Agaricus lemaneiformis',
 'Amaranth',
 'asparagus',
 'Baby Corn',
 'Bamboo shoots',
 'Basil',
 'Beef Tomato',
 'Bell pepper',
 'Big Chinese Cabbage',
 'Big cucumber',
 'Bok Choy',
 'Broccoli',
 'brocoli',
 'cabbage',
 'carrot',
 'celery',
 'chili',
 'Chinese Cabbage',
 'Chinese chives',
 'Chrysanthemum',
 'coriander',
 'corn',
 'cowpea',
 'Cucumber',
 'eggplant',
 'French beans',
 'Garlic',
 'Garlic sprouts',
 'ginger',
 'Green bamboo shoots',
 'green onion',
 'Green pepper',
 'Kale',
 'Lettuce',
 'Loofah',
 'Lotus root',
 'Mainland girl',
 'Momordica charantia',
 'Mountain Su',
 'Okra',
 'onion',
 'pea',
 'potato',
 'pumpkin',
 'rape',
 'Red broccoli',
 'Romaine',
 'Shallots',
 'spinach',
 'Sweet Pea',
 'sweet potato',
 'Sweet potato leaves',
 'Taro',
 'Water Lily',
 'Water spinach',
 'WaWa dishes',
 'White radish',
 'Winter melon',
 'Yam',
 'zucchini']

In [12]:
def load_image(data_path, img_size=224):
    X=[]
    y=[]
    label_list = os.listdir(data_path)
    label_num = np.arange(0,len(label_list))
    labels = dict(zip(label_list, label_num))
    
    for label in labels:
        path = Path(data_path)/label
        for img_name in os.listdir(path):
            img_path = Path(path,img_name)
            img = cv2.imread(img_path)
            print(img_path)
            if img is None:
                print(f'Skip {img_path} could not read image')
                continue 
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (img_size, img_size))
            X.append(img)
            y.append(labels[label])
    return np.array(X), np.array(y)

In [13]:
X_train, y_train = load_image(train_path)
X_train = X_train.astype('float32')/255.0

dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis1.jpeg
dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis10.jpeg
dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis103.jpg
dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis106.jpg
dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis107.jpg
dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis108.jpg
dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis109.jpg
dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis110.jpg
dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis112.jpg
dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis115.jpg
dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis117.jpg
dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis12.jpg
dataset_full_en\train\Agaricus lemaneiformis\Agaricus lemaneiformis120.jpeg
dataset_full_en\train\Agar

In [18]:
num_classes = len(os.listdir(train_path))
num_classes

batch_size = 32
img_size = (224, 224)

# Eğitim setini yükleme
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    image_size=img_size,
    batch_size=batch_size,
)
classes = train_ds.class_names
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

train_ds = train_ds.shuffle(buffer_size=1000)

train_size = int(0.8 * len(train_ds)) 
test_size = len(train_ds) - train_size

train_dataset = train_ds.take(train_size)
test_dataset = train_ds.skip(train_size)

print(classes)

Found 15067 files belonging to 60 classes.
['Agaricus lemaneiformis', 'Amaranth', 'Baby Corn', 'Bamboo shoots', 'Basil', 'Beef Tomato', 'Bell pepper', 'Big Chinese Cabbage', 'Big cucumber', 'Bok Choy', 'Broccoli', 'Chinese Cabbage', 'Chinese chives', 'Chrysanthemum', 'Cucumber', 'French beans', 'Garlic', 'Garlic sprouts', 'Green bamboo shoots', 'Green pepper', 'Kale', 'Lettuce', 'Loofah', 'Lotus root', 'Mainland girl', 'Momordica charantia', 'Mountain Su', 'Okra', 'Red broccoli', 'Romaine', 'Shallots', 'Sweet Pea', 'Sweet potato leaves', 'Taro', 'WaWa dishes', 'Water Lily', 'Water spinach', 'White radish', 'Winter melon', 'Yam', 'asparagus', 'brocoli', 'cabbage', 'carrot', 'celery', 'chili', 'coriander', 'corn', 'cowpea', 'eggplant', 'ginger', 'green onion', 'onion', 'pea', 'potato', 'pumpkin', 'rape', 'spinach', 'sweet potato', 'zucchini']


In [19]:
# 假設圖片大小是 224x224x3，視你的資料可調整
input_shape = (224, 224, 3)
base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_tensor=Input(shape=input_shape)
)

# 凍結所有 ResNet 層（先訓練 top）
for layer in base_model.layers:
    layer.trainable = False

# 建 top classifier
x = base_model.output
x = GlobalAveragePooling2D()(x)  # 平均池化
x = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)
output = Dense(60, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


In [ ]:
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# model.compile(
#     optimizer=optimizer,
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )

# # callbacks
# early_stop = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', patience=4, restore_best_weights=True
# )

# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6
# )

# checkpoint = tf.keras.callbacks.ModelCheckpoint(
#     'resnet_model_checkpoint.h5',
#     monitor='val_loss',
#     save_best_only=True,
#     save_weights_only=False,
#     verbose=1
# )


In [31]:
# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',  
    metrics=['accuracy']
)

# callbacks 建議多加 tensorboard
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
checkpoint = tf.keras.callbacks.ModelCheckpoint('resnet_model_checkpoint.h5', monitor='val_loss', save_best_only=True, verbose=1)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='./logs')

# Fit
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=20,
    callbacks=[early_stop, reduce_lr, checkpoint, tensorboard]
)



Epoch 1/20
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0434 - loss: 3.8976
Epoch 1: val_loss improved from inf to 3.89026, saving model to resnet_model_checkpoint.h5


376/376 ━━━━━━━━━━━━━━━━━━━━ 515s 1s/step - accuracy: 0.0434 - loss: 3.8976 - val_accuracy: 0.0441 - val_loss: 3.8903 - learning_rate: 1.0000e-04
Epoch 2/20
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 640ms/step - accuracy: 0.0476 - loss: 3.8831
Epoch 2: val_loss did not improve from 3.89026
376/376 ━━━━━━━━━━━━━━━━━━━━ 311s 814ms/step - accuracy: 0.0477 - loss: 3.8831 - val_accuracy: 0.0498 - val_loss: 3.9019 - learning_rate: 1.0000e-04
Epoch 3/20
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 781ms/step - accuracy: 0.0478 - loss: 3.8893
Epoch 3: val_loss improved from 3.89026 to 3.88263, saving model to resnet_model_checkpoint.h5


376/376 ━━━━━━━━━━━━━━━━━━━━ 375s 986ms/step - accuracy: 0.0478 - loss: 3.8893 - val_accuracy: 0.0480 - val_loss: 3.8826 - learning_rate: 1.0000e-04
Epoch 4/20
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 689ms/step - accuracy: 0.0457 - loss: 3.8896
Epoch 4: val_loss did not improve from 3.88263
376/376 ━━━━━━━━━━━━━━━━━━━━ 330s 860ms/step - accuracy: 0.0457 - loss: 3.8896 - val_accuracy: 0.0493 - val_loss: 3.8851 - learning_rate: 1.0000e-04
Epoch 5/20
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 780ms/step - accuracy: 0.0484 - loss: 3.8944
Epoch 5: val_loss improved from 3.88263 to 3.88247, saving model to resnet_model_checkpoint.h5


376/376 ━━━━━━━━━━━━━━━━━━━━ 412s 1s/step - accuracy: 0.0484 - loss: 3.8944 - val_accuracy: 0.0470 - val_loss: 3.8825 - learning_rate: 1.0000e-04
Epoch 6/20
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0450 - loss: 3.8947
Epoch 6: val_loss did not improve from 3.88247
376/376 ━━━━━━━━━━━━━━━━━━━━ 542s 1s/step - accuracy: 0.0450 - loss: 3.8947 - val_accuracy: 0.0533 - val_loss: 3.8868 - learning_rate: 1.0000e-04
Epoch 7/20
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0443 - loss: 3.8881
Epoch 7: val_loss did not improve from 3.88247
376/376 ━━━━━━━━━━━━━━━━━━━━ 541s 1s/step - accuracy: 0.0443 - loss: 3.8881 - val_accuracy: 0.0461 - val_loss: 3.8856 - learning_rate: 1.0000e-04
Epoch 8/20
376/376 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0452 - loss: 3.8919
Epoch 8: val_loss did not improve from 3.88247
376/376 ━━━━━━━━━━━━━━━━━━━━ 538s 1s/step - accuracy: 0.0452 - loss: 3.8919 - val_accuracy: 0.0523 - val_loss: 3.8916 - learning_rate: 5.0000e-05
Epoch 9/20
376/376 ━

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_dataset, batch_size=32)
val_generator = val_datagen.flow(test_dataset, batch_size=32)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop, reduce_lr, checkpoint]
)


In [ ]:
# 解凍 ResNet 最後幾層
for layer in base_model.layers[-30:]:
    layer.trainable = True

# 重新 compile 才能繼續訓練
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 接續訓練
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stop, reduce_lr]
)
